In [1]:
#Handle Imports
import cv2
import numpy as np
import math
import pylab as pl
import matplotlib.pyplot as plt
import random

from os import listdir
from os.path import isfile, join
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

## Collin 4/25
import os
import ipyparallel as ipp



from __future__ import print_function
np.random.seed(1337)  # for reproducibility

%matplotlib inline

Using TensorFlow backend.


In [2]:
import os
for filename in os.listdir("Positives/"):
        print(filename)
        if filename.endswith('.jpg'):
            newpath = 'Positives/' + filename 
            img = cv2.imread(newpath ,0)
            print(img)
            rows, cols = img.shape
        for x in range(4):
            if filename.endswith('.jpg'):				
                cv2.imwrite(os.path.join(os.path.join('Positives/', 'data'), filename  + '_' + str(x * 90) + '.jpg'),img)
                M = cv2.getRotationMatrix2D((cols/2, rows/2), 90, 1)
                img = cv2.warpAffine(img, M, (cols,rows))

81_yes.jpg
[[109 114  94 ...,  81 127  80]
 [111 117  96 ...,  86 109  79]
 [ 79  98 115 ..., 112  91  86]
 ..., 
 [116  80 107 ...,  74 106 103]
 [118  94  92 ...,  84  98  88]
 [106 110  87 ..., 105  98 114]]
36_yes.jpg
[[ 84  87 102 ...,  81  84  92]
 [ 89  90  93 ...,  90  83  87]
 [ 93  88  76 ..., 102  85  84]
 ..., 
 [ 64  64  91 ...,  88  86  90]
 [ 81  80  96 ...,  93  85  86]
 [ 95  91  93 ...,  95  84  85]]
205_yes.jpg
[[104  85  97 ...,  90  83  82]
 [ 90  88 112 ...,  96  85  94]
 [104  91  93 ...,  92  93 107]
 ..., 
 [116  63  77 ...,  87  97  84]
 [ 77 122  91 ...,  90  92  94]
 [112 105  83 ...,  82  93  87]]
4_03.jpg
[[126 118 119 ..., 135 135 121]
 [115 141 113 ..., 113 153 108]
 [118 119 119 ..., 133 131 130]
 ..., 
 [134 106 144 ..., 116 115 100]
 [112 131 149 ..., 111 134 128]
 [148 109 134 ..., 112 145 125]]
3_13.jpg
[[107  98  97 ...,  94  97  82]
 [ 72 111  87 ...,  98  95 107]
 [124  78  90 ...,  83 102  84]
 ..., 
 [126 100  89 ..., 103  92 114]
 [110 110  83

In [3]:
'''
This module will handle pre-processing such as labeling inages as positives and negatives.  If you would like to use
the data from the folders just download them and change the paths.
'''

#Read all images from folder.
pospath = "Positives/data/"
positives = [f for f in listdir(pospath) if isfile(join(pospath, f))]

negpath = "Negatives/"
negatives = [f for f in listdir(negpath) if isfile(join(negpath, f))]

#create array to hold all data
numImages = len(positives) + len(negatives)
data = np.zeros((numImages, 1025))

#choose 80/20 ratio for splitting data because of Pareto principle
split = math.floor(numImages * 0.8)

#load Positives
for i in range(len(positives)):
    path = pospath + positives[i]
    particle = cv2.imread(path )
    particle = cv2.cvtColor(particle, cv2.COLOR_BGR2GRAY)
    
    particle = particle.flatten()
    particle = np.insert(particle, 1024, 1)
    data[i] = particle
    
#load Negatives
for i in range(len(negatives)):
    path = negpath + negatives[i]
    particle = cv2.imread(path)
    particle = cv2.cvtColor(particle, cv2.COLOR_BGR2GRAY)
    
    particle = particle.flatten()
    particle = np.insert(particle, 1024, 0)
    data[len(positives) + i] = particle

#shuffle examples
np.random.shuffle(data)
data = data.astype('float32')
#split data into training and testing sets
X_train = data[int(0):int(split), int(0):int(1024)]
y_train = data[int(0):int(split), int(1024)]
X_test = data[int(split):,int(0):int(1024)]
y_test = data[int(split):, int(1024)]

#reshape images to be routed to classifier
X_train = np.reshape(X_train, (int(split), 32, 32))
X_test = np.reshape(X_test, (numImages - int(split), 32, 32))

In [4]:
X_train.shape[0]

720

In [5]:
'''This module will set up hyper parameters and  normalize the data'''

batch_size = 10
nb_classes = 2
nb_epoch = 80

# input image dimensions
img_rows, img_cols = 32, 32
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train.astype('int'), nb_classes)
Y_test = np_utils.to_categorical(y_test.astype('int'), nb_classes) 

X_train shape: (720, 32, 32, 1)
720 train samples
181 test samples


In [ ]:
'''Model creation and fitting'''

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])



/home/collin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="valid", input_shape=(32, 32, 1...)`
/home/collin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
/home/collin/anaconda2/lib/python2.7/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 720 samples, validate on 181 samples
Epoch 1/80
720/720 [==============================] - 3s - loss: 0.6491 - acc: 0.5861 - val_loss: 0.5153 - val_acc: 0.8895
Epoch 2/80
720/720 [==============================] - 2s - loss: 0.3863 - acc: 0.8333 - val_loss: 0.3346 - val_acc: 0.8895
Epoch 3/80
720/720 [==============================] - 2s - loss: 0.3172 - acc: 0.8417 - val_loss: 0.2917 - val_acc: 0.9006
Epoch 4/80
720/720 [==============================] - 2s - loss: 0.2855 - acc: 0.8653 - val_loss: 0.2340 - val_acc: 0.9061
Epoch 5/80
720/720 [==============================] - 3s - loss: 0.2591 - acc: 0.8750 - val_loss: 0.2162 - val_acc: 0.9558
Epoch 6/80
720/720 [==============================] - 3s - loss: 0.2212 - acc: 0.9125 - val_loss: 0.1408 - val_acc: 0.9669
Epoch 7/80
720/720 [==============================] - 3s - loss: 0.1830 - acc: 0.9292 - val_loss: 0.1534 - val_acc: 0.9503
Epoch 8/80
720/720 [==============================] - 3s - loss: 0.1460 - acc: 0.9528 - val_l

720/720 [==============================] - 2s - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0107 - val_acc: 0.9945
Epoch 67/80
720/720 [==============================] - 2s - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0089 - val_acc: 0.9945
Epoch 68/80
720/720 [==============================] - 2s - loss: 3.7062e-04 - acc: 1.0000 - val_loss: 0.0130 - val_acc: 0.9945
Epoch 69/80
720/720 [==============================] - 2s - loss: 0.0067 - acc: 0.9986 - val_loss: 0.0115 - val_acc: 0.9945
Epoch 70/80
720/720 [==============================] - 2s - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0094 - val_acc: 0.9945
Epoch 71/80
720/720 [==============================] - 2s - loss: 0.0024 - acc: 1.0000 - val_loss: 0.0092 - val_acc: 0.9945
Epoch 72/80
720/720 [==============================] - 2s - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0107 - val_acc: 0.9945
Epoch 73/80
720/720 [==============================] - 2s - loss: 5.2285e-04 - acc: 1.0000 - val_loss: 0.0100 - val_acc: 0.9945
Epoch 74/80


In [ ]:
'''Visualize Predictions'''

fig = plt.figure()
fig.subplots_adjust(hspace=1)

predictions = model.predict_classes(X_test, batch_size=32, verbose=0)

displays = 8

for i in range(displays):
    x = random.sample(xrange(1,len(X_test)), displays)
    img = np.squeeze(X_test[x[i]])
    ax = plt.subplot(341 + i)
    if(predictions[x[i]] == 1):
        ax.set_title("Particle")
    else:
        ax.set_title("Not a Particle")
    plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')

plt.show()

In [ ]:
predictions1 = model.predict_classes(X_test[0:1,int(0):int(1024)], batch_size=1, verbose=0)

In [ ]:
predictions1[0]

In [ ]:
import numpy as np
import sys
import cv2



class SlidingWindow(object):

    def __int__(self, blc, image, classifier, shift_r, shift_d):

        self.blc = blc
        self.image = image
        self.classifier = classifier
        self.shift_r = shift_r
        self.shift_d = shift_d

        """"
        BLC : Bottom Left Corner : This indicates the start point of the sliding window and takes in a value such as 32
        Image : This is the image that will be passed as a numpy array
        Classifier : This is the reference from the CNN handling the identification
        ShiftR : Shift Right : This is how many pixels the image will shift to the right
        ShiftD : Shift Down : This is how many pixels the image will shift down
        """


    def move_slider(self):

        count_x = 0  # number to increase each iteration of while
        count_y = 0
        shape_of_array = np.shape(self.image)  # the dimensions of the numpy Image array

        max_x = shape_of_array[1]  # the dimension along the x axis
        print( max_x)
        max_y = shape_of_array[0]  # the dimension along the y axis
        print( max_y)

        slice_obj = (slice(0, self.blc-1, 1), slice(0, self.blc-1, 1))  # sets the slice of 0 to the BLC of each axis
        sub_image = self.image[slice_obj]  # gets the pixel values from the 0 index to the max index of the passed BLC

        blc_x = self.blc
        blc_y = self.blc
        count_yes = 1
        dirname = 'Observed'

        while blc_y < max_y:

            # first run through starts at 0,0 and has a BLC of what was passed in i.e. 32x32
            # once that has run through this while loop lowers the sub_image(0, ShiftD)

            start_y = count_y * self.shift_d  # where the index of the array should start
            end_y = self.blc - 1 + (count_y * self.shift_d)  # where the index of the array for y should end

            new_y_range = (slice(start_y, end_y, 1))
            # new_y_range is the slice of the y axis from the start of the count to the BLC provided


            while blc_x < max_x:

                # as stated the first run through the top left sets to 0,0
                # in this case the window will slide to the right a number of pixels equal to shift_r

                # first iteration: (0, 0) through (31, 31)
                # second iteration: (shiftR, 0) through (31+ShiftR, 32)

                start_x = count_x * self.shift_r  # where the index of the array should start
                end_x = self.blc - 1 + (count_x * self.shift_r)  # where the index of the array for y should end

                new_x_range = (slice(start_x, end_x, 1))
                # new_x_dim is the slice of the y axis from the start of the count to the BLC provided

                slice_obj = (new_x_range, new_y_range)

                sub_image = self.image[slice_obj]

                c = classifier
                shape_of_sub_array = np.shape(sub_image)
                c.image=sub_image
                if(sub_image.shape == (32L, 32L) ):
                    myparticle = sub_image.flatten()
                    #myparticle = np.insert(myparticle, 1024, 1)
                    myparticle = myparticle.reshape(1,  img_rows, img_cols, 1)
                    myparticle = myparticle.astype('float32')
                    myparticle /= 255
                    
                    mypredictions = model.predict_classes(myparticle, batch_size=1, verbose=0)
                    if(mypredictions[0] == 1):
                        print('found one at X: ' ,start_x, 'and Y: ' , start_y)
                        file_name = '{}_yes.jpg'.format(count_yes)  # sets the name of each image to a new name inline with count_yes
                        cv2.imwrite(os.path.join(dirname, file_name), sub_image)  # writes the image to a new image with name file_name
                        count_yes += 1  # increments the count change the file name
                        
                    #if(mypredictions[0] == 0):
                        #print('found none at X: ',  start_x, 'and Y: ' ,start_y)
                        
                clone = self.image.copy()
                #cv2.rectangle(clone, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
                #cv2.imshow("Window", clone)
                #cv2.waitKey(1)
                #time.sleep(0.025)

                # output_string = "{}\n".format(c.c())
                # sys.stdout.write(output_string)
                # sys.stdout.flush()

                count_x += 1
                blc_x = self.blc + (count_x * self.shift_r)

            count_y += 1
            if blc_y < max_y:
                blc_y = self.blc + (count_y * self.shift_d)

            if blc_x >= max_x:
                blc_x = self.blc
                count_x = 0
                







In [ ]:
class classifier:

    def __int__(self, image):
        self.image = image


    def c(self):
        percent = 10
        prob = random.randrange(0,100)
        if prob > percent:
            return False
        else:
            return True

In [ ]:
'''class classifier:

    def __int__(self, image):
        self.image = image


    def c(self):
        predictions = model.predict_classes(X_test, batch_size=32, verbose=0)
        percent = 10
        prob = random.randrange(0,100)
        if prob > percent:
            return False
        else:
            return True'''

In [ ]:
import time
import random
import pprint
import numpy as np
import sys
import cv2
import os



class SlidingWindow(object):

    def __int__(self, blc, image, classifier, shift_r, shift_d, boundingboxes):

        self.blc = blc
        self.image = image
        self.classifier = classifier
        self.shift_r = shift_r
        self.shift_d = shift_d
        self.boundingboxes = boundingboxes

        """"
        BLC : Bottom Left Corner : This indicates the start point of the sliding window and takes in a value such as 32
        Image : This is the image that will be passed as a numpy array
        Classifier : This is the reference from the CNN handling the identification
        ShiftR : Shift Right : This is how many pixels the image will shift to the right
        ShiftD : Shift Down : This is how many pixels the image will shift down
        """


    def move_slider(self,verbose=0):

        count_x = 0  # number to increase each iteration of while
        count_y = 0
        shape_of_array = np.shape(self.image)  # the dimensions of the numpy Image array

        max_x = shape_of_array[1]  # the dimension along the x axis
        print( max_x)
        max_y = shape_of_array[0]  # the dimension along the y axis
        print( max_y)

        slice_obj = (slice(0, self.blc-1, 1), slice(0, self.blc-1, 1))  # sets the slice of 0 to the BLC of each axis
        sub_image = self.image[slice_obj]  # gets the pixel values from the 0 index to the max index of the passed BLC

        blc_x = self.blc
        blc_y = self.blc
        
        count_yes = 1
        dirname = 'Observed'
        

        while blc_y < max_y:

            # first run through starts at 0,0 and has a BLC of what was passed in i.e. 32x32
            # once that has run through this while loop lowers the sub_image(0, ShiftD)

            start_y = count_y * self.shift_d  # where the index of the array should start
            end_y = self.blc - 1 + (count_y * self.shift_d)  # where the index of the array for y should end

            new_y_range = (slice(start_y, end_y, 1))
            # new_y_range is the slice of the y axis from the start of the count to the BLC provided


            while blc_x < max_x:

                # as stated the first run through the top left sets to 0,0
                # in this case the window will slide to the right a number of pixels equal to shift_r

                # first iteration: (0, 0) through (31, 31)
                # second iteration: (shiftR, 0) through (31+ShiftR, 32)

                start_x = count_x * self.shift_r  # where the index of the array should start
                end_x = self.blc - 1 + (count_x * self.shift_r)  # where the index of the array for y should end

                new_x_range = (slice(start_x, end_x, 1))
                # new_x_dim is the slice of the y axis from the start of the count to the BLC provided

                slice_obj = (new_x_range, new_y_range)

                sub_image = self.image[slice_obj]

                c = classifier
                shape_of_sub_array = np.shape(sub_image)
                c.image=sub_image
                
                if(sub_image.shape == (32L, 32L) ):
                    
                    myparticle = sub_image.flatten()
                    #myparticle = np.insert(myparticle, 1024, 1)
                    myparticle = myparticle.reshape(1,  img_rows, img_cols, 1)
                    myparticle = myparticle.astype('float32')
                    myparticle /= 255
                    
                    
                    
                    mypredictions = model.predict_classes(myparticle, batch_size=1, verbose=0)
                    if(mypredictions[0] == 1):
                        if verbose == 1:
                            print('found one at X: ' ,start_x, 'and Y: ' , start_y)
                        
                        ## added 3/24 @ 4:45 pm - Matt
                        file_name = '{}_yes.jpg'.format(count_yes)  # sets the name of each image to a new name inline with count_yes
                        cv2.imwrite(os.path.join(dirname, file_name), sub_image)  # writes the image to a new image with name file_name
                        count_yes += 1  # increments the count change the file name
                        
                        ## added April 23rd @ 1:53 - Collin
                        self.boundingboxes.append([start_x,start_y,end_x,end_y])
                        
                        #cv2.rectangle(self.image, (start_x, start_y), (end_x, end_y), (0, 255, 0), 1) # shows a box aroudn the coordinates where it detects a particle
                    #if(mypredictions[0] == 0):
                    
                    
                        
                        
                clone = self.image.copy()
                #cv2.rectangle(clone, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
                #cv2.imshow("Window", clone)
                #cv2.waitKey(1)
                #time.sleep(0.025)

                # output_string = "{}\n".format(c.c())
                # sys.stdout.write(output_string)
                # sys.stdout.flush()

                count_x += 1
                blc_x = self.blc + (count_x * self.shift_r)

            count_y += 1
            if blc_y < max_y:
                blc_y = self.blc + (count_y * self.shift_d)

            if blc_x >= max_x:
                blc_x = self.blc
                count_x = 0
                
        ## added April 23rd @ 1:53 - Collin
        self.boundingboxes = np.array(self.boundingboxes)
    


'''


class Driver():

    def main(self):

        # image = np.random.rand(256, 256)
        # np.set_printoptions(precision=3, threshold=256, edgeitems=128, linewidth=256)
        # pprint.pprint(image)

        im = cv2.imread('1.jpg', 0)
        print( type(im))

        blc = 33
        c = classifier()
        c.image=im
        shift_r = 30
        shift_d = 30

        s_w = SlidingWindow()
        s_w.blc=blc
        s_w.image=im
        s_w.classifier=c
        s_w.shift_r=shift_r
        s_w.shift_d=shift_d

        s_w.move_slider()




if __name__ == '__main__':
    Driver().main()'''

In [ ]:
im = cv2.imread('1.jpg', 0)
print( type(im))

blc = 33
c = classifier()
c.image=im
shift_r = 7
shift_d = 7
s_w = SlidingWindow()
s_w.blc=blc
s_w.image=im
s_w.classifier=c
s_w.boundingboxes = []
s_w.shift_r=shift_r
s_w.shift_d=shift_d
s_w.move_slider()

In [ ]:
def non_maximum_supression_slow(boxes, overlapThresh):
# if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
 
    # initialize the list of picked indexes
    pick = []
 
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    
        # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list, add the index
        # value to the list of picked indexes, then initialize
        # the suppression list (i.e. indexes that will be deleted)
        # using the last index
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        suppress = [last]

        # loop over all indexes in the indexes list
        for pos in xrange(0, last):
            # grab the current index
            j = idxs[pos]

            # find the largest (x, y) coordinates for the start of
            # the bounding box and the smallest (x, y) coordinates
            # for the end of the bounding box
            xx1 = max(x1[i], x1[j])
            yy1 = max(y1[i], y1[j])
            xx2 = min(x2[i], x2[j])
            yy2 = min(y2[i], y2[j])

            # compute the width and height of the bounding box
            w = max(0, xx2 - xx1 + 1)
            h = max(0, yy2 - yy1 + 1)

            # compute the ratio of overlap between the computed
            # bounding box and the bounding box in the area list
            overlap = float(w * h) / area[j]

            # if there is sufficient overlap, suppress the
            # current bounding box
            if overlap > overlapThresh:
                suppress.append(pos)

        # delete all indexes from the index list that are in the
        # suppression list
        idxs = np.delete(idxs, suppress)

    # return only the bounding boxes that were picked
    return boxes[pick]

In [ ]:
#before

print ("[x] %d initial bounding boxes", (len(s_w.boundingboxes)))

pre_suppress = np.copy(s_w.image)
for x in s_w.boundingboxes:
    cv2.rectangle(pre_suppress, (x[1], x[0]), (x[3], x[2]), (0, 255, 0), 1)

In [ ]:
plt.figure(figsize=(26,14))
plt.imshow(pre_suppress,cmap = 'gray', interpolation = 'bicubic')

In [ ]:
pick = non_maximum_supression_slow(s_w.boundingboxes, 0.3)

In [ ]:
#after

print ("[x] %d final bounding boxes", (len(pick)))

post_suppress = np.copy(s_w.image)
for x in pick:
    cv2.rectangle(post_suppress, (x[1], x[0]), (x[3], x[2]), (0, 255, 0), 1)

In [ ]:
plt.figure(figsize=(26,14))
plt.imshow(post_suppress,cmap = 'gray', interpolation = 'bicubic')